# Load libraries

In [56]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
from torchvision import datasets, transforms, models
import pathlib

# Checking for device 

In [57]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Data loading

In [58]:
data_transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])
])

In [59]:
train_path='./seg_train/seg_train'
test_path='./seg_test/seg_test'

train_loader= DataLoader(datasets.ImageFolder(train_path,transform=data_transformer), batch_size=64, shuffle=True)
test_loader= DataLoader(datasets.ImageFolder(test_path,transform=data_transformer), batch_size=32, shuffle=True)

In [60]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


# CNN Network

In [61]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=6):
        super(ConvNet, self).__init__()

        # Output size after convolution filter
        # ((w - f + 2P) / s) + 1
        
        # Inpit shape = (256, 3, 150, 150)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # Shape (256, 12, 150, 150)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        # Shape (256, 12, 150, 150)
        self.relu1 = nn.ReLU()
        # Shape (256, 12, 150, 150)

        # Reudce the image size by factor of 2
        self.pool = nn.MaxPool2d(kernel_size=2)
        # Shape (256, 12, 75, 75)

        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        # Shape (256, 20, 75, 75)
        self.relu2 = nn.ReLU()
        # Shape (256, 20, 75, 75)

        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Shape (256, 32, 75, 75)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        # Shape (256, 32, 75, 75)
        self.relu3 = nn.ReLU()
        # Shape (256, 32, 75, 75)

        self.fc = nn.Linear(in_features=75 * 75 * 32, out_features=num_classes)

    # Feed forward function 
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
              
        output = self.pool(output)
              
        output = self.conv2(output)
        output = self.relu2(output)
              
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
                
        # Above output will be in matrix form, with shape (256,32,75,75)
            
        output = output.view(-1, 32 * 75 * 75)
              
        output = self.fc(output)
            
        return output

In [62]:
model = ConvNet(num_classes=6).to(device)

# Model training and saving best model

In [63]:
optimizer = Adam(model.parameters(), lr = 0.001, weight_decay = 0.0001)
loss_function = nn.CrossEntropyLoss()

In [64]:
num_epochs = 10

In [65]:
# Calculating the size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

print(train_count,test_count)

14034 3000


In [66]:
# Model training and saving best model
best_accuracy=0.0

for epoch in range(num_epochs):
    
    # Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss += loss.cpu().data * images.size(0)
        _,prediction = torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy / train_count
    train_loss=train_loss / train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        outputs = model(images)
        _,prediction = torch.max(outputs.data,1)
        test_accuracy += int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy / test_count
    
    
    print(f'Epoch: {str(epoch)} Train Loss: {str(train_loss)} Train Accuracy: {str(train_accuracy)} Test Accuracy: {str(test_accuracy)})
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    
       

Epoch: 0 Train Loss: tensor(6.1917) Train Accuracy: 0.5948411001852644 Test Accuracy: 0.699
Epoch: 1 Train Loss: tensor(2.0657) Train Accuracy: 0.7292290152486818 Test Accuracy: 0.6136666666666667
Epoch: 2 Train Loss: tensor(1.0190) Train Accuracy: 0.8032635029214764 Test Accuracy: 0.6883333333333334
Epoch: 3 Train Loss: tensor(0.5898) Train Accuracy: 0.8525010688328345 Test Accuracy: 0.7606666666666667
Epoch: 4 Train Loss: tensor(0.4228) Train Accuracy: 0.8858486532706284 Test Accuracy: 0.738
Epoch: 5 Train Loss: tensor(0.2176) Train Accuracy: 0.9335898532136241 Test Accuracy: 0.7576666666666667
Epoch: 6 Train Loss: tensor(0.1696) Train Accuracy: 0.9529713552800342 Test Accuracy: 0.767
Epoch: 7 Train Loss: tensor(0.1473) Train Accuracy: 0.956890409006698 Test Accuracy: 0.7466666666666667
Epoch: 8 Train Loss: tensor(0.1180) Train Accuracy: 0.9641584722816018 Test Accuracy: 0.7353333333333333
Epoch: 9 Train Loss: tensor(0.1600) Train Accuracy: 0.9551090209491235 Test Accuracy: 0.7013333